In [1]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import AnimalShelter


# Importing AnimalShelter class 
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
animals = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(animals.read({}))



#df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
# create dash application


app = JupyterDash('Dylan Ackron CS 340')

image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Dylan Ackron Project 2 Dashboard'))),
    html.Hr(),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Allows 10 rows per page, and starts on page 0. Columns and Rows may be selected but not deleted/edited
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
       
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
       
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
       
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(animals.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
   
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(debug=False)

read method


 * Running on http://127.0.0.1:30957/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2024 02:45:20] "GET /_alive_33485834-a3db-4c02-b9af-8176de84f870 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:30957/


127.0.0.1 - - [15/Oct/2024 02:45:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 02:45:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 02:45:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 02:45:27] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Oct/2024 02:45:27] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Oct/2024 02:45:27] "POST /_dash-update-component HTTP/1.1" 200 -


read method


...

127.0.0.1 - - [15/Oct/2024 03:08:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:16] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:16] "POST /_dash-update-component HTTP/1.1" 200 -


read method


127.0.0.1 - - [15/Oct/2024 03:08:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:19] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:23] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:25] "POST /_dash-update-component HTTP/1.1" 200 -


read method


127.0.0.1 - - [15/Oct/2024 03:08:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2024 03:08:32] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Oct/2024 03:08:32] "POST /_dash-update-component HTTP/1.1" 200 -
